# Part 3: Algorithmic Optimizations

We made the model train faster, but now we want to improve it's performance. Here, we will try to use the hyperparameters used in GPT-2 or GPT-3 papers to train our model, and see if we get a better performance.


In [ ]:
# Imports
import torch
import time


1. Betas in Adam: $\beta _1 = 0.9, \beta _2=0.95, \epsilon = 10^{-8} $
2. Clipping the global norm of the gradients to 1. Gradient clipping is generally did to handle the problem of exploding gradients. The intuitive reason for this is that if you get a bad batch of data, your loss will be high, and thus gradients will be high also, which you don't want to reflect in the model weights. If the norm of your gradients are above some fixed $c$, then you clip their values at $c$. We do it by: $\frac{g}{||g||}$. In GPT-3, the global norm was clipped at 1. If norm is increasing / you get a sudden spike, things are bad / unstable (in intiali few iterations, the norm can be very high, which is fine ).

I'm not entirely sure about two things, which I need to clarify with someone:
- Are we clipping each parameter tensor individually, or concatenating all parameter gradients into one big tensor and then clipping it? Most likely, we're taking the *global* norm- by concatenating all parameter gradients.
- When we have models like GPT-2, which has so many parameters, if we're clipping the global norm to 1, wouldn't the weight updates be *really* small because we will be multiplying these scaled gradients again by the learning rate? If they are, why can we use lesser precision- it should cause problems?


In [ ]:
# ... model initialization as before. Here, I am initializing to None because I have to copy a
# lot of code, which is useless. So init as previous notebooks.
model = None
train_loader = None
device = 'cpu'

# Only optimizer and training loop changes
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-4)

for i in range(50):
    t0 = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()
    with torch.autocast(device_type=device, dtype=torch.bfloat16):
        logits, loss = model(x, y)

    loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping

    optimizer.step()
    torch.cuda.synchronize()

    t1 = time.time() 

    dt = t1 - t0
    tokens_processed = train_loader.B * train_loader.T

    tokens_per_sec = tokens_processed / dt

    print(f"Step {i:4d} | Loss: {loss.item():.6f} | norm: {norm:.4f} | dt: {dt*1000:.2f}ms | tok/sec: {tokens_per_sec}")



3. GPT-3 uses a cosine decay learning rate scheduler with warm up.